In [3]:
from datetime import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.nddata import CCDData, Cutout2D
import astropy.units as u
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.table import Table
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import ccdproc
import photutils

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

from mmtwfs.wfs import wfs_norm
from skycam_utils.photometry import make_background, make_segmentation_image, load_bright_star_catalog
from skycam_utils.pipeline import process_image

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%cd /Users/tim/MMT/sky_brightness/asi_skycam/Bias/

/Volumes/LaCie 8TB/sky_brightness/asi_skycam/Bias


In [ ]:
biases = ccdproc.ImageFileCollection('.', keywords="*", glob_include="Bias*")

In [ ]:
for h in biases.hdus(save_location='reduced', overwrite=True, ccd_kwargs={'unit': 'adu', 'gain': 1.0}):
    h.header['fixscale'] = True
    h.data = h.data / 16

In [ ]:
scaled_biases = ccdproc.ImageFileCollection('./reduced', keywords="*", glob_include="Bias_*")

In [ ]:
bias = ccdproc.combine(
    list(scaled_biases.ccds(ccd_kwargs={'unit': 'adu'})),
    method='average',
    mem_limit=1e9,
    clip_extreme=True,
    nlow=2,
    nhigh=2,
    dtype=np.float64
)
bias.write("./bias.fits", overwrite=True)

In [6]:
%cd ../Light

/Volumes/LaCie 8TB/sky_brightness/asi_skycam/Light


In [ ]:
filters = ['clear', 'u', 'g', 'r', 'i']
#filters = ['u']
for f in filters:
    rpath = f"{f}/reduced"
    if not Path.exists(Path(rpath)):
        Path.mkdir(Path(rpath))
    ic = ccdproc.ImageFileCollection(f"./{f}", keywords="*", glob_include="Light_*")
    for h in ic.hdus(save_location=rpath, overwrite=True, ccd_kwargs={'unit': u.adu, 'gain': 1.0}):
        h.header['fixscale'] = True
        h.data = h.data / 16
        h.data = h.data - bias.data

In [ ]:
test_im = CCDData.read(Path("./r/reduced/Light_024.fits"), unit=u.adu)
test_im.header['FILTER']

In [ ]:
Cutout2D(test_im, (300, 300), (100, 100), copy=True).data

In [9]:
g_cat = process_image("./g/reduced/Light_024.fits")